In [221]:
import itertools
import numpy as np
import plotly.express as plx
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d

from function_splitting import crop_axis, form_x_y_polygons
from functions_voronoi import voronoi_finite_polygons_2d, voronoi_polygons

np.set_printoptions(suppress=True)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [373]:
INPUT_WIDTH = 100
INPUT_HEIGHT = 40

in_points = np.array([[x, y] for x in range(INPUT_WIDTH) for y in range(INPUT_HEIGHT)])
in_points.shape

(4000, 2)

In [378]:
N_points = 10

kmeans = KMeans(n_clusters=N_points, random_state=0)
clusters = kmeans.fit_predict(in_points)
new_points = np.array([np.mean(in_points[clusters == curr], axis=0) for curr in set(clusters)])
new_points += np.array([[x_r, y_r] for x_r,y_r in zip(np.random.randint(N_points, size=N_points), 
                                                      np.random.random(N_points)+np.random.randint(N_points, size=N_points))])
new_points.shape

(10, 2)

In [398]:
polygons = voronoi_polygons(new_points)
cropped_polygons = crop_axis(polygons, range_x=(0, INPUT_WIDTH), range_y=(0, INPUT_HEIGHT))
x_polygs, y_polygs = form_x_y_polygons(cropped_polygons)

fig = go.Figure(go.Scatter(x=x_polygs, y=y_polygs, mode='lines+markers'))
fig.show()

# test skew at line

In [399]:
test_polygon = cropped_polygons[0]
test_polygon

array([[ 84.2484,  40.    ],
       [ 85.8227,  26.1104],
       [100.    ,  28.8883],
       [100.    ,  40.    ],
       [ 84.2484,  40.    ]])

In [400]:
test_polygon[0] in test_polygon

True

In [401]:
center_polygon = test_polygon.mean(axis=0)

save_percent = 0.5
a_p, c_p, b_p = test_polygon[0], center_polygon, test_polygon[1]
t = 1-save_percent

a1_p = (1-t)*a_p + t*c_p
b1_p = (1-t)*c_p + t*b_p


points = np.array([
    test_polygon[0],
    a1_p, 
    b1_p,
    test_polygon[1]
])

path = evaluate_bezier(points, 20)
x, y = points[:,0], points[:,1]
px, py = path[:,0], path[:,1]

fig = plx.line(x=px, y=py)
fig.add_traces(
    list(plx.scatter(x=x, y=y).select_traces())
)

# test skew at polygon

In [402]:
from Bezier_functions import skew_polygon, skew_graph

import warnings
warnings.filterwarnings('ignore')

In [403]:
skewed_polygon = skew_polygon(test_polygon)

x, y = test_polygon[:, 0], test_polygon[:, 1]
px, py = skewed_polygon[:, 0], skewed_polygon[:, 1]

fig = plx.line(x=px, y=py)
fig.add_traces(
    list(plx.scatter(x=x, y=y).select_traces())
)

In [408]:
polygons = skew_graph(cropped_polygons)
x_polygs, y_polygs = form_x_y_polygons(polygons)

fig = go.Figure(go.Scatter(x=x_polygs, y=y_polygs, mode='lines'))
fig.show()